## Importing the libraries

In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from matplotlib.image import imread

import seaborn as sns

import os

import cv2

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix

from tensorflow.keras.utils import to_categorical

from keras.models import Sequential

from tensorflow.keras.layers import Dense,Dropout,Activation

from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.optimizers import Adam

## 加載圖片，並轉換成灰色

In [ ]:
def load_data(data_dir):
    images = []
    labels = []
    for i in range(10):
        folder = os.path.join(data_dir, str(i))
        for filename in os.listdir(folder):
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
                labels.append(i)
    return np.array(images), np.array(labels)

data_dir = '/kaggle/input/corrupted-mnist/mnist'
images, labels = load_data(data_dir)

## 切分成訓練資料、測試資料，9:1的比例

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.1, stratify = labels, random_state = 3)


**顯示train dataset及test dataset的筆數**

In [ ]:
print('train data records =', len(X_train))

print('test data records =', len(X_test))

**顯示train dataset中image和label的格式**

In [ ]:
print('x_train image format =', X_train.shape)

print('y_train label format =', len(y_train.shape))

## 隨機顯示訓練資料的9張圖片

In [ ]:
def display_rand_images(images, labels):
    plt.figure(1 , figsize = (19 , 10))
    n = 0 
    for i in range(9):
        n += 1 
        r = np.random.randint(0 , images.shape[0] , 1)
        
        plt.subplot(3 , 3 , n)
        plt.subplots_adjust(hspace = 0.3 , wspace = 0.3)
        plt.imshow(images[r[0]])
        
        plt.title('Number : {}'.format(labels[r[0]]))
        plt.xticks([])
        plt.yticks([])
        
    plt.show()
    
display_rand_images(X_train, y_train)

In [ ]:
print('X_train:', X_train.shape)

print('y_train:', y_train.shape)

In [ ]:
X_train = X_train.reshape(54000, 1024).astype('float32')

X_test = X_test.reshape(6000, 1024).astype('float32')

print('X_train:', X_train.shape)

print('X_test:', X_test.shape)

In [ ]:
print(X_train[0])

print(X_test[0])

## 資料標準化

In [ ]:
X_train_normalize = X_train/255

X_test_normalize = X_test/255

print(X_train_normalize[0])

print(X_test_normalize[0])


In [ ]:
y_TrainOneHot = to_categorical(y_train)

y_TestOneHot = to_categorical(y_test)

y_TrainOneHot[:5]

## 建立模型

In [ ]:
model=Sequential()

# kernel_initializer=’normal’ → 使用常態分的亂數來初始化weight權重及bias偏差。
# activation=’relu’ → 激活函數，指定使用relu
model.add(Dense(units=256, input_dim=1024, kernel_initializer='normal', activation='relu'))

model.add(Dense(units=128, kernel_initializer='normal', activation='relu'))

model.add(Dense(units=64, kernel_initializer='normal', activation='relu'))

# 使用Dropout防止過度擬合
model.add(Dropout(0.25))

#輸出層
model.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

In [ ]:
# 選擇損失函數、優化方法及成效衡量方式
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## 模型訓練

In [ ]:
train_history = model.fit(x=X_train_normalize, y=y_TrainOneHot, validation_split=0.2, epochs=20, batch_size=200, verbose=2)

## 評估訓練結果

In [ ]:
def show_train_history(train_history, train, validation):

    plt.plot(train_history.history[train])

    plt.plot(train_history.history[validation])

    plt.title('Train History')

    plt.ylabel('train')

    plt.xlabel('Epoch')

    plt.legend(['train', 'validation'], loc='upper left')

    plt.show()
    
#  顯示準確率圖表
show_train_history(train_history, 'accuracy', 'val_accuracy')
#  顯示loss誤差圖表
show_train_history(train_history, 'loss', 'val_loss')

## 使用未參與訓練的測試資料測試，模型準確率

In [ ]:
scores = model.evaluate(X_test_normalize,y_TestOneHot,verbose=0)

print('Accuracy = ', scores[1])

## 混淆矩陣

In [ ]:
y_pred = np.argmax(model.predict(X_test_normalize), axis=-1)
y_true = np.argmax(y_TestOneHot, axis=-1)
confusion_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,7))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()